# Data Challenge 1: Scrape and Crawl
This is the Jupyter Notebook for the first Data Challenge of Winter 2018 INTEG 475.

## Group Members
Q Chen

Amy Lu

Kevin Zhang

## Beginner Challenge
Pick a single registered charity and write a series of functions to scrape:
1. Schedule 6 financial data
2. data on directors/trustees and like officials, and
3. data on employee compensation

Write the output to a series of .csv files. Your notebook should include text / comments that explain the logic of each
substantial chunk of code. If you don’t explain what you are doing, and why you made the decisions you did, you will not get full marks.

## Part 1

We begin scraping our chosen charity by looking at the Schedule 6 financial data.
First we store the URL of the Schedule 6 data as a string into a variable, `sched6_url`.

In [60]:
# Schedule 6 - Financial Data
sched6_url = "http://www.cra-arc.gc.ca/ebci/haip/srch/t3010form23sched6-eng.action?b=118983329RR0001&fpe=2016-06-30&n=KITCHENER+ROTARY+CLUB+CHARITABLE+FOUNDATION&r=http%3A%2F%2Fwww.cra-arc.gc.ca%3A80%2Febci%2Fhaip%2Fsrch%2Ft3010form23-eng.action%3Fb%3D118983329RR0001%26amp%3Bfpe%3D2016-06-30%26amp%3Bn%3DKITCHENER%2BROTARY%2BCLUB%2BCHARITABLE%2BFOUNDATION"

As the financial data of Schedule 6 is stored in HTML tables, we will use pandas to efficiently scrape these tables.

In [61]:
import pandas as pd

We set up our scraping by creating a function called `clean_up`. This function consumes a string that represents an HTML URL, and outputs a list of cleaned pandas dataframes. The function begins by calling the pandas function `read_html`, which reads the HTML at a given URL, finds all tables in the HTML, converts those tables into dataframes, and stores the dataframes into a list. If there are no tables in the HTML, then the function will print the error message to the screen. Assuming there are tables in the HTML, the list of dataframes is passed through a `for` loop, which takes the `Amount` column of every HTML table and replaces all non-digit and non-period characters (such as $) with empty strings, essentially removing those characters. The `for` loop is able to do this with the pandas `replace` function, which can enable RegEx to ignore (`[^]`) digits (`\d`) and periods (`\.`) when replacing characters. 

In [62]:
def clean_up(link):
    try:
        df = pd.read_html(link)
    except ValueError as error:
        print("Error: {0}".format(error))
        print("There are no tables on this page.")
        return None
    for i in df:
        i['Amount'] = i['Amount'].replace('[^\d^\.]', '', regex=True)
    return df

We run the function `clean_up` on `sched6_url`, and store the output list of dataframes into `df_list`.

In [63]:
df_list = clean_up(sched6_url)

By calling each dataframe in the list, we know that the first dataframe is Assets, second is Liabilities, third is Revenue, and fourth is Expenditures. We found that the Expenditures dataframe had turned the `Line number` column values into floats due to `NaN` values, and so we used a pandas function `fillna` to replace the `NaN` values with empty strings, ultimately removing the float-value issue.

In [64]:
df_list[3] = df_list[3].fillna('')

Finally, we exported the dataframes into csv files so we can process or share the data externally if we want.

In [65]:
df_list[0].to_csv('assets.csv', index = False)
df_list[1].to_csv('liabilities.csv', index = False)
df_list[2].to_csv('revenue.csv', index = False)
df_list[3].to_csv('expenditures.csv', index = False)

## Part 2
Next we will scrape the data on directors, trustees, and like officials. We will begin like Part 1 by storing the Section B URL as a string variable.

In [66]:
# Section B - data on directors/trustees and like officials
sectionB_url = "http://www.cra-arc.gc.ca/ebci/haip/srch/t3010form23officers-eng.action?b=118983329RR0001&fpe=2016-06-30&n=KITCHENER+ROTARY+CLUB+CHARITABLE+FOUNDATION"

This part requires pandas, BeautifulSoup, and urllib.request, we have to import BeautifulSoup now (as we imported pandas earlier).

In [67]:
from bs4 import BeautifulSoup
import urllib.request

Using BeautifulSoup, we will create a function `html_to_dict` that takes in a URL string and outputs a list of dictionaries. The function begins by using BeautifulSoup to take in the HTML of the given URL link, and then finds all the `ul` tags which have `class_` that are `"list-unstyled"`.

We loop through each `ul` tag (which is every other item in the pseudo-list `ul_text`) to find all `li` tags, which is where the relevant information is stored. Each `ul` tag stores the information for one person, which is why we are looping through it, and each person's information will be stored in their own dictionary. The `range` for the `for` loop specifies a start index position of 0 and an end index position of 15, looping only for every other item (indicated by the 2). This is because every odd numbered item does not contain the desired information, and coding the `range` this way allows us to skip the items we don't want, and because there are only 8 people listed on this page. 

We know that the Full Name, Start Date, and other information are found at certain `li` tags, and so we extract the strings from the appropriate `li` tags by finding where the information is, after the expected colon symbol. We add the attribute as keys and the information as values to the dictionary `d`, which is later appended to the dictionary list `dict_ls`. The function finally returns the list of dictionaries, or the list of people profiles. 

In [68]:
def html_to_dict(link):
    page = urllib.request.urlopen(link)
    soup = BeautifulSoup(page, 'html.parser')
    
    dict_ls = []
    ul_text = soup.find_all("ul", class_="list-unstyled")
    t = ""
    colon_pos = 0
    
    for i in range(0,15,2):
        d = {}
        li_text = ul_text[i].find_all("li")
        
        t = li_text[0].text
        colon_pos = t.find(":")
        d["Full Name"] = li_text[0].text[colon_pos+2:]
        
        t = li_text[2].text
        colon_pos = t.find(":")
        d["Start Date"] = li_text[2].text[colon_pos+2:]
        
        t = li_text[3].text
        colon_pos = t.find(":")
        d["End Date"] = li_text[3].text[colon_pos+2:]
        
        t = li_text[4].text
        colon_pos = t.find(":")
        d["Position"] = li_text[4].text[colon_pos+2:]
        
        t = li_text[5].text
        colon_pos = t.find(":")
        d["At arm's length with other Directors"] = li_text[5].text[colon_pos+2:]
        
        dict_ls.append(d)
    
    return dict_ls

We are now ready to call the `html_to_dict` function on `sectionB_url`. The function returns a list of dictionaries which is then converted into a Pandas data frame.

In [69]:
directors_df = pd.DataFrame.from_dict(html_to_dict(sectionB_url))
directors_df

At arm's length with other Directors    End Date          Full Name  \
0                                  Yes  2017-06-30       Ray G Taylor   
1                                  Yes  2017-06-30      Martin  Jones   
2                                  Yes  2016-12-31    Brent  DeKoning   
3                                  Yes  2017-06-30       David  Smith   
4                                  Yes  2017-06-30  David  Martindale   
5                                  Yes  2017-06-30      Karen  Redman   
6                                  Yes  2017-06-30      John  Webster   
7                                  Yes  2017-06-30      Cheryl  Ewing   

    Position  Start Date  
0  Treasurer  2004-07-01  
1      Chair  2010-07-01  
2   Director  2015-07-01  
3   Director  2015-07-01  
4   Director  2015-07-01  
5   Director  2012-07-01  
6   Director  2016-07-01  
7  Secretary  2013-07-01

The data frame columns are not ordered intuitively. To order the columns, we create an ordered list containing the names of the columns in our desired order. We then pass this list to the Pandas data frame subsetting syntax to reorder the columns.

In [70]:
idx = ["Full Name", "Position", "Start Date", "End Date", "At arm's length with other Directors"]
directors_df = directors_df[idx]
directors_df

Full Name   Position  Start Date    End Date  \
0       Ray G Taylor  Treasurer  2004-07-01  2017-06-30   
1      Martin  Jones      Chair  2010-07-01  2017-06-30   
2    Brent  DeKoning   Director  2015-07-01  2016-12-31   
3       David  Smith   Director  2015-07-01  2017-06-30   
4  David  Martindale   Director  2015-07-01  2017-06-30   
5      Karen  Redman   Director  2012-07-01  2017-06-30   
6      John  Webster   Director  2016-07-01  2017-06-30   
7      Cheryl  Ewing  Secretary  2013-07-01  2017-06-30   

  At arm's length with other Directors  
0                                  Yes  
1                                  Yes  
2                                  Yes  
3                                  Yes  
4                                  Yes  
5                                  Yes  
6                                  Yes  
7                                  Yes

Finally, we export the reordered data frame to a CSV file (ignoring the data frame indices).

In [71]:
directors_df.to_csv("directors.csv",index=False)

## Part 3
Finally, we will look at the data on employee compensation. We stored the Schedule 3 URL as a string variable.

In [72]:
# Schedule 3 - Data on Employee Compensation
sched3url = "http://www.cra-arc.gc.ca/ebci/haip/srch/t3010form23sched3-eng.action?b=118983329RR0001&fpe=2016-06-30&n=KITCHENER+ROTARY+CLUB+CHARITABLE+FOUNDATION&r=http%3A%2F%2Fwww.cra-arc.gc.ca%3A80%2Febci%2Fhaip%2Fsrch%2Ft3010form23-eng.action%3Fb%3D118983329RR0001%26amp%3Bfpe%3D2016-06-30%26amp%3Bn%3DKITCHENER%2BROTARY%2BCLUB%2BCHARITABLE%2BFOUNDATION"

Upon accessing the Schedule 3 page for our chosen charity, we found that the charity has not reported any data on employee compensation, and so unfortunately we won't be able to scrape anything here :(

## Appendix

### Self-Assessments

Q: This data challenge was a good review of the web scraping methods we learned from the readings and the exercises we did in class. I found that having some background in Python (CS 115 and 116, plus some additional work with ArcPy for GIS) really helped me understand the syntax and methods we used as a group to tackle this challenge. For this challenge, I wrote the explanations for how Part 1 worked, and also broke down how the pandas `read_html` function and Regex worked for our group's understanding. For Part 2, I helped debug the `html_to_dict` function to find that we only wanted the information from every other `ul` tag, and pushed for a way to reorder the columns in the pandas dataframe so that the `directors.csv` would make more sense. I felt more comfortable doing the Beginner challenge versus the Advanced one due to being rusty in Python and that learning how to use Pandas and BeautifulSoup took longer understanding on my part. I felt that I pushed myself in terms of learning the different functions and capabilities that the pandas package had, and also in working with a small group of people with widely different backgrounds. This web scraping challenge allowed me to utilize and review almost all of my Python knowledge, such as how to work with lists, dictionaries, strings, and for loops. By the end of the challenge, I realized how important Pandas was to the process (and learned a lot of it through trial and error), and realized how a good understanding of how web pages are structured can change the way you look at the data that BeautifulSoup parses and returns back to you. 

Amy: I wrote and tested code for writing the output of the main function in Part 2 to CSV. I helped write textual comments and debug/refactorize code in for both parts. I learned how to apply prior Python knowledge in a "real-world" context. It was exciting to work with dictionaries in an applicable setting for the first time, and design its structure to achieve desired results. I also learned how to condense code through iterative refinements. For example, I spent a lot of time mocking up "dummy" dictionaries and playing with the `pd.DataFrame.from_dict` function. It was rewarding to see it get condensed into one line. I pushed myself to explore different ways of coding (for example, trying to use abstract functions instead of loops) and pick the more elegant solution. For the future, I want to become more comfortable with Beautiful Soup and web crawling. I am still on the initial learning curve for scrapping of web data in terms of project design and understanding available methodologies, and I'd like to perform more interesting data analyses.

Kevin: My contribution for this data challenge was the coding aspect of the data challenge. Even though I have some knowledge of Python from CS 116, I still found this assignment to be a little challenging at first. I feel like I don't really have good grasp with pandas, beautifulsoup ,etc and I don't know how to "use" them properly. The example notebooks that were posted were extremely useful in getting started. Looking for the relevant information on a website's page source was overwhelming, very confusing and there were many times where I wanted to just give up. I had to push myself outside my comfort zone and understand HTML which I do not have any experience in. When I finally understood how lists and tags in HTML worked, I was able to scrape tables and then export as csv (part 1) and implement a loop to extract the relevant information in the list tags and store them into dictionaries (part 2). Through this assignment, I learned what libraries to use depending on what task I am working on. In addition, I have a better understanding of how to scrape the web. My goal for the upcoming data challenges are to attempt the advanced challenge.